In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re

Read in manifest to get unique case IDs

In [5]:
#tsv file is manifest file downloaded from GDC portal
manifest = 'TCGA/gdc_sample_sheet.2019-11-15.tsv'

tsv_file= manifest
csv_table=pd.read_csv(tsv_file,sep='\t')
csv_table.to_csv('TCGA/gdc_sample_sheet.2019-11-15.csv',index=False)
df = pd.read_csv('TCGA/gdc_sample_sheet.2019-11-15.csv')
#df.head()

#remove double case ids and create a case id list
for i in range(0,len(df)):
    if len(df['Case ID'][i]) > 12:
        df['Case ID'][i] = df['Case ID'][i][0:12]
case_ids = df['Case ID'].unique()
case_ids = case_ids[0:2].tolist()
case_ids

['TCGA-GS-A9TQ', 'TCGA-GS-A9U3']

The following cell uses filters to extract file IDs based on specified field, value combinations and then. The filters are split into two sections, one for each data type.
The file IDs are then passed through the GDC server and the set of requested files is downloaded automatically.

In [6]:
files_endpt = "https://api.gdc.cancer.gov/files"

filters={
    "op":"or",
    "content":[
        {
        "op":"and",
        "content":[
            {
                "op":"in",
                "content":{
                    "field":"cases.submitter_id",
                    "value":case_ids
                }
            },
            {
                "op":"in",
                "content":{
                    "field":"files.data_type",
                    "value":"Slide Image"
                }
            }
        ]
        },
        {
        "op":"and",
        "content":[
            {
                "op":"in",
                "content":{
                    "field":"cases.submitter_id",
                    "value":case_ids
                }
            },
            {
                "op":"in",
                "content":{
                    "field":"files.data_type",
                    "value":"Gene Expression Quantification"
                }
            }
        ]
        }
    ]
}

# Here a GET is used, so the filter parameters should be passed as a JSON string.

params = {
    "filters": json.dumps(filters),
    "fields": "file_id",
    "format": "JSON",
    "size": "1000"
    }

response = requests.get(files_endpt, params = params)

file_uuid_list = []

# This step populates the download list with the file_ids from the previous query
for file_entry in json.loads(response.content.decode("utf-8"))["data"]["hits"]:
    file_uuid_list.append(file_entry["file_id"])

data_endpt = "https://api.gdc.cancer.gov/data"

params = {"ids": file_uuid_list}

response = requests.post(data_endpt, data = json.dumps(params), headers = {"Content-Type": "application/json"})

response_head_cd = response.headers["Content-Disposition"]

file_name = re.findall("filename=(.+)", response_head_cd)[0]

with open(file_name, "wb") as output_file:
    output_file.write(response.content)

Still need to sort out annotated mutation files

In [1]:
%load_ext watermark

%watermark -v -p numpy,pandas,requests,json,re

CPython 3.7.3
IPython 7.6.1

numpy 1.17.3
pandas 0.24.2
requests 2.22.0
json 2.0.9
re 2.2.1
